In [34]:
import os
import pickle

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [2]:
def load_pickles(folder_path):
    all_data = {} 

    for filename in os.listdir(folder_path):
        if filename.endswith('.pkl'):  # Check if the file is a pickle file
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Unpickle the file and store its contents in the dictionary
                data = pickle.load(file)
                all_data[filename] = data

    return all_data

In [3]:
folder_path = '_results_fl_val'
val_data = load_pickles(folder_path)
val_data.keys()

dict_keys(['mlp_fl_val_res.pkl', 'tft_val_res.pkl', 'transformer_fl_val_res.pkl', 'xgboost_fl_val_res.pkl'])

In [4]:
import sys
sys.path.append(r'C:\Users\obhlivoj\DP\System-Imbalance-Forecasting\models\xgboost')

import torch
import json

from train import get_ds
from config import get_config

In [6]:
val_src, test_src = [], []
for k in range(8):    
    cfg = get_config()
    cfg['tgt_step'] = k
    cfg['path_pickle'] = '../data/data_TS/'

    cfg["exo_vars"] = ['total_load']
    cfg["forward_vars"] = ["most_recent_forecast_load"]

    _, val_scl, test_scl = get_ds(cfg, return_raw=True)

    src_val, src_test = [], []
    for dt in val_scl:
        src_val.append(dt['x_input'].numpy())
    for dt in test_scl:
        src_test.append(dt['x_input'].numpy())

    val_src.append(np.stack(src_val))
    test_src.append(np.stack(src_test))

In [28]:
meta_models_cat = []
meta_forest_cat = []
for k in range(8):    
    preds_mlp = val_data['mlp_fl_val_res.pkl']['preds'][k].squeeze().cpu().numpy()
    preds_tft = val_data['tft_val_res.pkl']['preds'][:,k]
    preds_tf = val_data['transformer_fl_val_res.pkl']['preds'][:,k].squeeze().numpy()
    preds_xgb = val_data['xgboost_fl_val_res.pkl']['preds'][k]

    y_true = val_data['mlp_fl_val_res.pkl']['gt'][k].squeeze().cpu().numpy()
    min_size = np.min((preds_mlp.shape[0], preds_tft.shape[0], preds_tf.shape[0], preds_xgb.shape[0]))

    stacked_preds = np.column_stack((preds_mlp[:min_size], preds_tft[:min_size], preds_tf[:min_size], preds_xgb[:min_size]))
    # lr
    meta_model = LinearRegression()
    meta_model.fit(stacked_preds, y_true[:min_size])
    meta_models_cat.append(meta_model)
    # rf
    param_grid = {
    'n_estimators': [64, 128, 256, 512],
    'max_depth': [2, 3, 4],
    }
    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    tscv = TimeSeriesSplit(n_splits=5)
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=tscv, n_jobs=-1, refit=True)
    #grid_search.fit(np.concatenate((stacked_preds, val_src[k][:min_size]), axis=1), y_true[:min_size]) 3. method
    grid_search.fit(stacked_preds, y_true[:min_size])

    meta_forest_cat.append(grid_search.best_estimator_)
    # meta_forest.fit(stacked_preds, y_true[:min_size]) # 1. method

    # meta_forest = RandomForestRegressor(n_estimators=80, max_depth=12, random_state=69) # 2. method
    # meta_forest.fit(np.concatenate((stacked_preds, val_src[k][:min_size]), axis=1), y_true[:min_size])
    # meta_forest_cat.append(meta_forest)

In [29]:
for k, mod in enumerate(meta_forest_cat):
    print(f'{k}: max_depth: {mod.max_depth}, n_estimators: {mod.n_estimators}')

0: max_depth: 4, n_estimators: 64
1: max_depth: 4, n_estimators: 256
2: max_depth: 3, n_estimators: 64
3: max_depth: 3, n_estimators: 512
4: max_depth: 3, n_estimators: 256
5: max_depth: 3, n_estimators: 64
6: max_depth: 3, n_estimators: 128
7: max_depth: 4, n_estimators: 64


In [30]:
folder_path = '_results_fl'
unpickled_data = load_pickles(folder_path)
unpickled_data.keys()

dict_keys(['mlp_fl_test_res.pkl', 'tft_test_res.pkl', 'transformer_fl_test_res.pkl', 'xgboost_fl_test_res.pkl'])

In [39]:
def print_results(method: str = "simple", meta_models_cat = None):
    rmse = {}
    mae = {}
    r2 = {}

    for k in range(8):
        preds_mlp = unpickled_data['mlp_fl_test_res.pkl']['preds'][k].squeeze().cpu().numpy()
        preds_tft = unpickled_data['tft_test_res.pkl']['preds'][:,k]
        preds_tf = unpickled_data['transformer_fl_test_res.pkl']['preds'][:,k].squeeze().numpy()
        preds_xgb = unpickled_data['xgboost_fl_test_res.pkl']['preds'][k]

        y_true = unpickled_data['mlp_fl_test_res.pkl']['gt'][k].squeeze().cpu().numpy()

        min_size = np.min((preds_mlp.shape[0], preds_tft.shape[0], preds_tf.shape[0], preds_xgb.shape[0]))

        if method == "simple":
            res = (preds_mlp[:min_size] + preds_tft[:min_size] + preds_tf[:min_size] + preds_xgb[:min_size]) / 4
        elif method == "lm":
            stacked_predictions = np.column_stack((preds_mlp[:min_size], preds_tft[:min_size], preds_tf[:min_size], preds_xgb[:min_size]))
            res = meta_models_cat[k].predict(stacked_predictions)
        elif method == "rf":
            stacked_predictions = np.column_stack((preds_mlp[:min_size], preds_tft[:min_size], preds_tf[:min_size], preds_xgb[:min_size]))
            #res = meta_models_cat[k].predict(np.concatenate((stacked_predictions, test_src[k][:min_size]), axis=1))
            res = meta_models_cat[k].predict(stacked_predictions)

        rmse[k] = mean_squared_error(y_true[:min_size], res, squared=False)
        mae[k] = mean_absolute_error(y_true[:min_size], res)
        r2[k] = r2_score(y_true[:min_size], res)

    print("Model\tRMSE\tMAE\tR2")
    for k in range(8):
        print(
            f'{k+1}\t{rmse[k]:.2f}\t{mae[k]:.2f}\t{r2[k]:.2f}')


In [32]:
print_results('simple')

Model	RMSE	MAE
1	109.39	80.21
2	130.98	94.79
3	141.01	101.02
4	146.40	103.74
5	151.24	107.16
6	154.41	109.05
7	155.91	109.82
8	155.52	109.50


In [40]:
print_results('lm', meta_models_cat)

Model	RMSE	MAE	R2
1	107.55	79.40	0.57
2	130.07	95.33	0.38
3	140.31	101.49	0.27
4	146.14	104.20	0.21
5	151.17	107.56	0.16
6	154.33	109.40	0.12
7	155.58	110.06	0.11
8	154.70	109.71	0.12


In [33]:
print_results('rf', meta_forest_cat)

Model	RMSE	MAE
1	109.04	80.14
2	131.19	96.22
3	141.63	102.33
4	147.21	104.74
5	151.90	107.77
6	154.89	109.58
7	155.91	110.33
8	155.72	110.42


Another ensemble was set up that could improve the resulting prediction based on models using future lags. In order not to skew the results, only data from the original validation set was used to train the ensemble model and tested on the original test data. 

We refer to a system that uses the outputs of other models as an ensemble. By combining other models, the ensemble should then achieve better prediction accuracy. In general, any kind of model can be used as an ensemble, but less complex ones are commonly used. For further information on ensemble models, we refer the reader to X.

Several ensemble models have been proposed to improve the prediction accuracy. In addition to a simple averaging method, linear regression and decision trees (CITE) were also implemented. In addition to the predictions from the 4 models themselves (MLP, XGBoost, TFT and Transformer), the input data to the ensembles were added to the original models. In the case of decision trees, a grid search was also implemented to find the best hyperparameters.

In the end, the best ensemble model turned out to be the linear model, which uses only the predictions from the original models as predictors. The more complex random forest struggled with overfitting despite hyperparameter optimization. Due to the fact that for ensemble models the data used for training is as large as the test data, the good generalization ability of the model is complicated.

